In [1]:
import os
os.chdir('..')

In [ ]:
from datetime import timezone
from pathlib import Path

import polars as pl
from dateutil import parser
from datetime import timedelta

from config import DataFrequency, BINANCE_DATA_DIR, TradeType

from util.ts_manager import TSManager
from aws.kline.parse import read_csv
from aws.client_async import AwsKlineClient

pl.Config.set_tbl_rows(30)


In [ ]:
symbol = 'BTCUSDT'
time_interval = '1m'
symbol_kline_dir = BINANCE_DATA_DIR / 'parsed_data' / TradeType.spot.value / 'klines' / symbol / time_interval

mgr = TSManager(symbol_kline_dir)

df1 = mgr.read_all()
df1.shape

In [ ]:
api_dir = BINANCE_DATA_DIR / "api_data" / TradeType.spot.value / "klines" / symbol / time_interval
ldf2 = pl.scan_parquet(api_dir)

ldf = pl.concat([df1.lazy(), ldf2])
ldf = ldf.unique("candle_begin_time", keep="last").sort("candle_begin_time")

df = ldf.collect()
df.shape